<a href="https://colab.research.google.com/github/TONNY-01/My_Insurance_Fraud_Detection./blob/main/My_Insurance_Fraud_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT THIS DATA IS OBTAINED FROM KAGGLE.
import kagglehub
arpan129_insurance_fraud_detection_path = kagglehub.dataset_download('arpan129/insurance-fraud-detection')

print('Data source import complete.')


In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


In [ ]:
data = pd.read_excel('/kaggle/input/insurance-fraud-detection/Worksheet in Case Study question 2.xlsx')
data.head()

In [ ]:
data.to_csv("Insurance.csv", index=False)

In [ ]:
df = pd.read_csv('Insurance.csv')

In [ ]:
df.isnull().sum()

In [ ]:
df['policy_number'].nunique()

In [ ]:
df.duplicated()

In [ ]:
df1 = df.drop(columns=['policy_number', 'policy_bind_date', 'policy_state', 'policy_csl', 'policy_deductable', 'policy_annual_premium'], axis=1)

In [ ]:
df2 = df1.drop(columns=['incident_date', 'incident_city', 'incident_state'], axis=1)
df2

In [ ]:
df2['incident_location'].head()

In [ ]:
import numpy as np

# Replace "?" with NaN
df2.replace("?", np.nan, inplace=True)

In [ ]:
df2.isnull().sum()

In [ ]:
# Define a list of non-sense values to replace
nonsense_values = ["unknown", "none", "n/a"]

# Replace them with NaN
df2.replace(nonsense_values, np.nan, inplace=True)


In [ ]:
df2.isnull().sum()

In [ ]:
df2.drop('incident_location', axis=1, inplace=True)

In [ ]:
df2.drop(columns=['auto_make', 'auto_model'], axis=1, inplace=True)

In [ ]:
# check the imbalace of the dataset
import matplotlib.pyplot as plt

target_counts = df['fraud_reported'].value_counts()

target_counts.plot(kind='pie', autopct='%1.1f%%', colors=['skyblue', 'salmon'])
plt.title('Class Distribution of Fraud Reported')
plt.ylabel('')
plt.show()

print(target_counts)

In [ ]:
df2.drop('umbrella_limit', axis=1, inplace=True)

In [ ]:
df2['authorities_contacted'].unique()

In [ ]:
df2['authorities_contacted'].fillna('None', inplace=True)

In [ ]:
df2['police_report_available'].unique()

In [ ]:
df2['property_damage'].unique()

In [ ]:
# Fill missing values
df2['collision_type'].fillna('Unknown', inplace=True)
df2['property_damage'].fillna('No', inplace=True)
df2['police_report_available'].fillna('No', inplace=True)


In [ ]:
df2.isnull().sum()

In [ ]:
df2.shape

In [ ]:
numerical_columns = df.select_dtypes(include=['number']).columns.tolist()
numerical_columns

In [ ]:
# Plot box and whisker plots for numerical columns
numerical_columns = df.select_dtypes(include=['number']).columns

for column in numerical_columns:
    plt.figure(figsize=(8, 4))
    plt.boxplot(df[column].dropna(), vert=False, patch_artist=True, boxprops=dict(facecolor='skyblue'))
    plt.title(f'Box and Whisker Plot for {column}')
    plt.xlabel(column)
    plt.show()


In [ ]:
df2.head()

In [ ]:
ordinal_column = ['insured_education_level', 'incident_severity']
nonordinal_columns = ['insured_sex', 'insured_occupation', 'insured_hobbies', 'insured_relationship',  'incident_type', 'collision_type',
                     'authorities_contacted', 'property_damage', 'police_report_available',
                     'fraud_reported']

In [ ]:
df2['insured_education_level'].unique()

In [ ]:
df2['incident_severity'].unique()

In [ ]:
# fit_transform
# inverse_transform


from sklearn.preprocessing import LabelEncoder

label_encoders = {}

for col in ['insured_education_level', 'incident_severity']:
  le = LabelEncoder()
  df2[f'{col}_encoded'] = le.fit_transform(df2[col])
  label_encoders[col] = dict(zip(le.classes_, le.transform(le.classes_)))


label_encoders

In [ ]:
df2.columns

In [ ]:
df2.drop(columns=['incident_severity', 'insured_education_level'], axis=1, inplace=True)

In [ ]:
nonordinal_columns = ['insured_sex', 'insured_occupation', 'insured_hobbies', 'insured_relationship',  'incident_type', 'collision_type',
                     'authorities_contacted', 'property_damage', 'police_report_available',
                     'fraud_reported']

In [ ]:
df2 = pd.get_dummies(df2, columns=nonordinal_columns, drop_first=True)

In [ ]:
df2.head()

In [ ]:
df2.head()

In [ ]:
df2.columns

In [ ]:
df2 = df2.replace({True: 1, False: 0})

In [ ]:
# I am going to do undersampling in order to tackle the issue of imbalanced dataset
X = df2.drop('fraud_reported_Y', axis=1)
y = df2['fraud_reported_Y']
from imblearn.under_sampling import RandomUnderSampler

rus = RandomUnderSampler(sampling_strategy=1)

X_res, y_res = rus.fit_resample(X,y)

ax = y_res.value_counts().plot.pie(autopct='%.2f')
_ = ax.set_title('Under_Sampling')

In [ ]:
# I try all the available models
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.preprocessing import StandardScaler


X_train, X_test, y_train, y_test = train_test_split(X_res, y_res, test_size=0.3, random_state=42)

# Standardize the data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# List of classifiers to evaluate
models = [
    ('Random Forest', RandomForestClassifier()),
    ('Logistic Regression', LogisticRegression(max_iter=1000)),
    ('SVM', SVC()),
    ('Decision Tree', DecisionTreeClassifier()),
    ('K-Nearest Neighbors', KNeighborsClassifier()),
    ('Naive Bayes', GaussianNB()),
    ('Gradient Boosting', GradientBoostingClassifier()),
    ('AdaBoost', AdaBoostClassifier())
]

# Evaluate each model
for name, model in models:
    model.fit(X_train_scaled, y_train)
    y_pred = model.predict(X_test_scaled)
    print(f"\n{name} Classification Report:")
    print(classification_report(y_test, y_pred))


In [ ]:
# explore to find the best possible model using cross-validation
from sklearn.model_selection import cross_val_score

best_model = None
best_score = 0

for name, model in models:
    scores = cross_val_score(model, X, y, cv=5, scoring='accuracy')
    mean_score = np.mean(scores)
    print(f"{name} Mean Accuracy: {mean_score}")

    if mean_score > best_score:
        best_score = mean_score
        best_model = model


best_model.fit(X_train_scaled, y_train)
y_pred = best_model.predict(X_test_scaled)
print(f"\nBest Model: {best_model.__class__.__name__}")
print(classification_report(y_test, y_pred))
